In [ ]:
from pathlib import Path
from icefabric_manage import build, load_table_from_catalog
from icefabric_tools import to_geopandas

In [4]:
warehouse_path = "../data/warehouse"
catalog = build(
    Path("/Users/taddbindas/projects/NGWPC/icefabric/data/parquet"),
    catalog_settings = {
        'type': 'sql',
        "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

Table network already exists, loading it
Table nexus already exists, loading it
Table flowpath-attributes already exists, loading it
Table divides already exists, loading it
Table pois already exists, loading it
Table flowpath-attributes-ml already exists, loading it
Table divide-attributes already exists, loading it
Table flowpaths already exists, loading it
Table hydrolocations already exists, loading it
Table lakes already exists, loading it


In [5]:
load_table_from_catalog(catalog, "network")

AttributeError: 'NoneType' object has no attribute 'load_table'